# Advanced Matplotlib Plotting

By the end of this tutorial, you'll know how to produce and customize a number of different graphics types using Matplotlib.

## Required Preparation

 - Go over the [Lecture 3](https://robertsj.github.io/me400_notes/lectures/Basic_Data_Processing_with_NumPy_and_Matplotlib.html) and [Lecture 4](https://robertsj.github.io/me400_notes/lectures/More_on_NumPy_Arrays.html) notes from ME 400 on Matplotlib (and NumPy)
 - Skim the examples at the [Matplotlib Gallery](https://matplotlib.org/3.1.1/gallery/index.html)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Consider this very basic but "correct" plot:

In [ ]:
x = np.linspace(0, 10)
np.random.seed(1234)
y = 1 + 2*x + np.random.normal(loc=0.0, scale=1.0, size=len(x))
z = np.polyval(np.polyfit(x, y, 1), x)
plt.figure(1)
plt.plot(x, y, 'o', x, z)
plt.legend(['$y$', '$z$'])
plt.xlabel('$x$')
plt.savefig('example1.png')

Is there anything wrong with this image?

## Customizing Plots

With  `matplotlib.rcParams`, we can customize many aspects of plots:
    

In [ ]:
def init_nice_plots() :
    from matplotlib import rcParams
    rcParams['font.family'] = 'serif'
    rcParams['font.serif'] = ['Times']
    rcParams['xtick.direction'] = 'out'
    rcParams['ytick.direction'] = 'out'
    rcParams['xtick.labelsize'] = 18
    rcParams['ytick.labelsize'] = 18
    rcParams['ytick.left'] = True
    rcParams['ytick.right'] = True
    rcParams['ytick.left'] = True
    rcParams['ytick.right'] = True        
    rcParams['lines.linewidth'] = 1
    rcParams['axes.labelsize']  = 20
    #rcParams['axes.facecolor']  = 'white'
    rcParams['legend.fontsize'] = 16
    rcParams['figure.autolayout'] = True
    rcParams['text.usetex'] = True

In [ ]:
init_nice_plots()
plt.figure(2)
plt.plot(x, y, 'o', x, z)
plt.legend(['$y$', '$z$'])
plt.xlabel('$x$')
plt.savefig('example2.png')

In [ ]:
plt.figure(3)
plt.plot(x, y, 'o', x, z)
plt.legend(['$y$', '$z$'])
plt.xlabel('$x$')
plt.savefig('example3.pdf')

## Displaying Errors

One approach: error bars, or

In [ ]:
import scipy.stats
pe = scipy.stats.t.ppf(0.95, len(x)-2)*\
     np.sqrt(sum((y - z)**2)/len(y))

plt.figure(4)
plt.plot(x, y, 'o')
plt.errorbar(x, z, yerr=pe)

An alternative: bands, or

In [ ]:
import scipy.stats
z_low = z-pe
z_high = z+pe
plt.figure(5)
plt.plot(x, y, 'o')
plt.plot(x, z, '--')
plt.plot(x, z_low, 'r')
plt.plot(x, z_high, 'r')
# can we shade the region between the solid lines?

## Visualizing 2-D Data

In [ ]:
n = 100
x = np.linspace(0, 1, n)
y = np.linspace(0, 1,n)
xx, yy = np.meshgrid(x, y)
f = np.sin(xx*yy)

# compute gradient, but on small grid
n = 12
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
xxx, yyy = np.meshgrid(x, y)
dfdx = yyy*np.cos(xxx*yyy)
dfdy = xxx*np.cos(xxx*yyy)

In [ ]:
# plot both
plt.figure(6)
plt.contourf(xx, yy, f, cmap='jet')
plt.quiver(xxx, yyy, dfdx, dfdy, color='white')
plt.xlabel('x')
plt.ylabel('y')
plt.axis([0,1,0,1])
plt.savefig('two_d_jet.png')

In [ ]:
# plot both
plt.figure(7)
plt.contour(xx, yy, f, 10)
plt.streamplot(xxx, yyy, dfdx, dfdy)
plt.xlabel('x')
plt.ylabel('y')
plt.axis([0,1,0,1])

**A challenge!**  Consider the following contour plot (from D.S. McGregor et al. NIM A 343 (1994)):

<img src="mcgregor.png" alt="drawing" width="400"/>


In [ ]:
def Q(rho_e, rho_h) :
    return rho_e + rho_e**2*(np.exp(-1.0/rho_e)-1.0) + \
           rho_h + rho_h**2*(np.exp(-1.0/rho_h)-1.0)
    
def sig_Q(rho_e, rho_h) :
    a = rho_e**2 + 2.*rho_e**3*(np.exp(-1.0/rho_e)-1) + \
        0.5*rho_e**3*(1-np.exp(-2.0/rho_e))
    b = rho_h**2 + 2.*rho_h**3*(np.exp(-1.0/rho_h)-1) + \
        0.5*rho_h**3*(1-np.exp(-2.0/rho_h))
    c = 2.*rho_e*rho_h + 2.*rho_e**2*rho_h*(np.exp(-1.0/rho_e)-1) + \
        2.*rho_h**2*rho_e*(np.exp(-1.0/rho_h)-1)
    d = 2.*(rho_e*rho_h)**2/(rho_e-rho_h)*(np.exp(-1.0/rho_e)-np.exp(-1.0/rho_h))
    return np.sqrt( a+b+c+d-Q(rho_e,rho_h)**2)
   
def R(rho_e, rho_h) :
    return 100*sig_Q(rho_e, rho_h)/Q(rho_e, rho_h)

In [ ]:
n = 100
H = np.logspace(-2, 2, n)
E = np.logspace(-2, 2, n) 

H, E = np.meshgrid(H, E, sparse=False, indexing='ij')
res = R(E, H)

plt.figure(1, figsize=(6,6))
plt.contour(np.log10(E), np.log10(H), res, colors='k')
plt.axis('equal')


Things that need fixin':

   - *appropriate axis labels* (e.g., `Electron Extraction Factor')
   - correct contour levels (i.e., 0.1, 0.2, 0.5, 1%, and so on).
     Hint: look up the documentation for `plt.contour`.
   - *correct $x$  and $y$ tick values* (e.g., -1 should be 0.1 and 2
       should be 100). Hint: look up, e.g.,`plt.xticks`.
   - *annotations for each contour line*. Hint: look up 
       `plt.text`, paying specific attention to `fontsize`,
      `horizontalalignment`, and `verticalalignment`. 
       You might also which to consider using `scipy.optimize.newton`
       to help you automatically find where text should be located,
       e.g., you know that the upper-left 40\% box should be located where 
       $F(x) = 100 - R(\rho_e, 100) = 0$.  However, you may simply
       place each text annotation manually.
   - *logarithmic minor tick marks*.   Note the 
       original has minor tick marks spaced logarithmically, whereas
       my solution has no minor tick marks.  Hint: look 
       up `plt.gca().yaxis`.